In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module simple_controls


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-ad279118'


loading module save_resampled
loading module selection
loading module bsa_proba
loading module modelGraphs
loading module profilewindow
loading module ana_image_math
loading module paletteViewer
loading module foldsplit
loading module anacontrolmenu
loading module gradientpalette
loading module palettecontrols
loading module meshsplit
loading module volumepalettes
loading module gltf_io
loading module infowindow
loading module histogram
loading module statsplotwindow
loading module valuesplotwindow
all python modules loaded
Anatomist started.


In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
import umap
import os

## Embeddings and their associated z-score (after MOSTEST) for different regions of interest

### SC-sylv_left

In [96]:
ukb_SC_sylv_left_embeddings = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/SC-sylv_left/14-00-57_116/ukb_random_epoch80_embeddings/full_embeddings.csv', index_col=0)
SC_sylv_left_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/SC-sylv_left_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### SC-sylv_right

In [97]:
ukb_SC_sylv_right_embeddings = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/SC-sylv_right/13-19-08_28/ukb_random_epoch80_embeddings/full_embeddings.csv', index_col=0)
SC_sylv_right_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/SC-sylv_right_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### CINGULATE_left

In [98]:
ukb_cingulate_left_embeddings = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/CINGULATE_left/2024-07-16/16-58-04_1/ukb_random_epoch80_embeddings/full_embeddings.csv', index_col=0)
cingulate_left_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/CINGULATE_left_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### CINGULATE_right

In [99]:
ukb_cingulate_right_embeddings = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/CINGULATE_right/09-52-22_75/ukb_random_epoch80_embeddings/full_embeddings.csv', index_col=0)
cingulate_right_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/CINGULATE_right_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### FIP_left

In [100]:
ukb_FIP_left_embeddings = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/FIP_left/14-56-46_3/ukb_random_epoch110_embeddings/full_embeddings.csv', index_col=0)
FIP_left_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/FIP_left_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### FIP_right

In [101]:
ukb_FIP_right_embeddings = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/FIP_right/20-16-33_2/ukb_random_epoch120_embeddings/full_embeddings.csv', index_col=0)
FIP_right_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/FIP_right_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### Orbital_left

In [102]:
# pas de traçabilité, à refaire
#ukb_orbital_left_embeddings = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/ORBITAL_left/20-56-02_1/ukb_random_embeddings/full_embeddings.csv', index_col=0)
#orbital_left_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/ORBITAL_left_mostest_all_chr.most_orig.zmat.csv', sep='\t')


### Orbital_right

In [103]:
ukb_orbital_right_embeddings = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/ORBITAL_right/15-53-41_191//ukb_random_epoch80_embeddings/full_embeddings.csv', index_col=0)
orbital_right_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/ORBITAL_right_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### STi-STs-STpol_left

In [104]:
ukb_STi_STs_STpol_left = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/STi-STs-STpol_left/14-30-34_34/ukb_random_epoch190_embeddings/full_embeddings.csv', index_col=0)
STi_STs_STpol_left_z_score = pd.read_csv('/volatile/ad279118/Irene/MOSTEST/Results/STi-STs-STpol_left_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### Region name

In [109]:
region_name = 'ORBITAL_right'
path_to_most_results = '/volatile/ad279118/Irene/MOSTEST/Results/'
z_score = pd.read_csv(f'{path_to_most_results}/{region_name}_mostest_all_chr.most_orig.zmat.csv', sep='\t')

### Get the direction given by the SNPs z-scores

In [111]:
print(ukb_orbital_right_embeddings.shape)
ukb_orbital_right_embeddings.head()

(21045, 256)


,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
ID,,,,,,,,,,,,,,,,,,,,,
sub-1000021,8.359276,1.393321,-3.196660,-11.687957,-7.284793,-5.947208,-6.337971,-24.955900,-4.806345,23.253164,...,-12.953685,-10.156114,-27.988750,-8.481975,5.088031,9.323348,5.608405,13.522653,-16.861488,-6.155326
sub-1000458,-10.749617,17.492110,-7.229668,1.425336,5.559165,-8.395197,-16.463188,8.947845,-13.438421,-21.016268,...,-3.147220,0.267443,-17.167963,25.359734,10.749844,-9.823949,-47.673664,-7.540023,-11.120076,10.969814
sub-1000575,-13.732792,10.635822,-16.314667,-14.749505,-4.846186,-21.462553,3.404566,-8.007508,-17.625782,13.084105,...,-48.863144,-17.198288,-13.846172,-17.697680,16.121006,-10.284886,14.381026,-7.883124,5.451983,0.468994
sub-1000606,-13.519534,1.969375,-0.363315,-1.145881,-5.049136,-19.041105,3.229115,-0.416106,-20.166530,-29.480904,...,-19.120272,16.428568,22.130460,-22.099056,-17.620304,-3.753097,-16.484900,-15.712536,18.070097,15.770590
sub-1000963,-13.752464,-1.547562,-0.544788,-8.198496,-5.528333,-29.882793,10.143282,-8.406162,-14.920433,6.493918,...,-44.661774,-15.098756,-10.772803,-10.961065,5.243499,6.560422,4.186872,-2.910577,3.421086,2.256803


In [117]:
orbital_right_z_score = orbital_right_z_score[['CHR','SNP','PVAL']+[f'dim{i}' for i in range(1,ukb_orbital_right_embeddings.shape[1]+1)]]
orbital_right_z_score.iloc[:,:8].head()

,CHR,SNP,PVAL,dim1,dim2,dim3,dim4,dim5
0,15,rs202109518,1.406139e-13,-1.422264,0.664990,-1.038796,0.677525,0.311259
1,15,rs200294259,4.076472e-36,-0.639176,-0.735111,0.528111,0.985628,0.616616
2,15,rs578030041,3.150767e-09,-0.146279,2.079909,0.451031,-1.870035,0.341744
3,15,rs376848849,3.150767e-09,-0.146279,2.079909,0.451031,-1.870035,0.341744
4,15,rs3933731,4.076859e-13,0.139025,-0.035373,-0.891899,-0.585226,-0.409859


In [115]:
orbital_right_z_score[orbital_right_z_score.SNP=='rs200294259']

,CHR,SNP,PVAL,dim1,dim2,dim3,dim4,dim5,dim6,dim7,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
1,15,rs200294259,4.076472e-36,-0.639176,-0.735111,0.528111,0.985628,0.616616,-2.342429,-0.694866,...,-0.725193,0.731048,2.184578,-2.537933,0.364947,-1.652185,-0.490197,-1.050762,3.219553,2.221781


In [114]:
orbital_right_z_score.dim150.hist()

<AxesSubplot:>

## Visualization (example of CINGULATE_right for now)

In [26]:
scaler = StandardScaler()
scaler.fit(ukb_cingulate_right_embeddings)

scl_bdd_ukb = scaler.transform(ukb_cingulate_right_embeddings)

reducer2D = umap.UMAP(n_components=2)
reducer2D.fit(scl_bdd_ukb)
bdd_2D_UKB = reducer2D.transform(scl_bdd_ukb)

/casa/home/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [27]:
bdd_2D_UKB = pd.DataFrame(bdd_2D_UKB, columns=['Dim 1', 'Dim 2'])
bdd_2D_UKB['Dataset'] = 'UkBioBank'
bdd_2D_UKB['ID'] = ukb_cingulate_right_embeddings.index
bdd_2D_UKB.head()

,Dim 1,Dim 2,Dataset,ID
0,6.828441,2.267577,UkBioBank,sub-1000021
1,1.688752,4.747731,UkBioBank,sub-1000458
2,3.698661,6.495202,UkBioBank,sub-1000575
3,5.779747,4.328494,UkBioBank,sub-1000606
4,0.072204,3.114254,UkBioBank,sub-1000963


In [28]:
subject_id_list = []
dataset_name_list = []

In [30]:
# Create the scatter plot using plotly express
fig = px.scatter(
    bdd_2D_UKB, x='Dim 1', y='Dim 2', 
    color='Dataset',
    title='2D UMAP HCP and UKB',
    labels={'0': 'dim 1', '1': 'dim 2'},
    hover_data= ['Dataset', 'ID'],
    opacity=0.5,
    width=800, height=600
)

#fig.show()

# Convert the figure to a FigureWidget
f = go.FigureWidget(fig)

# Define the callback function
def click_callback(trace, points, selector):
    for trace_index in range(len(f.data)):
        if trace_index == points.trace_index:
            customdata = f.data[trace_index].customdata
            for i in points.point_inds:
                point_dataset, point_id = customdata[i]
                print(f"Clicked point ID: {point_id}, Dataset: {point_dataset}")
                subject_id_list.append(point_id)
                dataset_name_list.append(point_dataset)

# Attach the callback to the on_click event for all traces
for trace in f.data:
    trace.on_click(click_callback)

# Display the figure widget
f

FigureWidget({
    'data': [{'customdata': array([['UkBioBank', 'sub-1000021'],
                                   ['UkBioBank', 'sub-1000458'],
                                   ['UkBioBank', 'sub-1000575'],
                                   ...,
                                   ['UkBioBank', 'sub-6023847'],
                                   ['UkBioBank', 'sub-6024038'],
                                   ['UkBioBank', 'sub-6024754']], dtype=object),
              'hovertemplate': ('Dataset=%{customdata[0]}<br>Di' ... '{customdata[1]}<extra></extra>'),
              'legendgroup': 'UkBioBank',
              'marker': {'color': '#636efa', 'opacity': 0.5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'UkBioBank',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'd4922623-e0b5-4f91-a5f4-e89777b5a383',
              'x': array([6.8284407, 1.6887518, 3.6986613, ..., 4.7936697, 1.4355619, 1.1729393],
               

Clicked point ID: sub-5194119, Dataset: UkBioBank
Clicked point ID: sub-1458453, Dataset: UkBioBank
Clicked point ID: sub-4565650, Dataset: UkBioBank
Clicked point ID: sub-2881581, Dataset: UkBioBank
Clicked point ID: sub-4030931, Dataset: UkBioBank
Clicked point ID: sub-1705902, Dataset: UkBioBank
Clicked point ID: sub-2054042, Dataset: UkBioBank
Clicked point ID: sub-1128000, Dataset: UkBioBank
Clicked point ID: sub-2761606, Dataset: UkBioBank
Clicked point ID: sub-5442840, Dataset: UkBioBank
Clicked point ID: sub-1871487, Dataset: UkBioBank


In [31]:
side = "R"
region = "CINGULATE" #"ORBITAL" #"CINGULATE"

bucket_files = []

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops'

    if dataset.lower() in ['hcp']:
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset.lower()}/crops/2mm/{region}/mask/{side}crops'

    filename = f'{path}/{subject_id}_cropped_skeleton.nii.gz'#.minf'

    if os.path. isfile(filename):
        bucket_files.append(filename)
    else:
        print(f"{filename} is not a correct path, or the .bck doesn't exist")
bucket_files

from soma import aims

def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

block = a.createWindowsBlock(3) # nb of columns
d = {}

for i, file in enumerate(bucket_files):
    d[f'bck_{i}'] = to_bucket(a.loadObject(file))
    d[f'w_{i}'] = a.createWindow('3D', block=block)#geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
    d[f'w_{i}'].addObjects(d[f'bck_{i}'])

dic_vol = {}
dim = 0
rep = 0
while dim == 0 and rep < len(subject_id_list):
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset_name_list[rep]}/crops/2mm/{region}/mask/{side}crops"
    if os.path. isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
        sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
        dim = sum_vol.shape
        sum_vol.fill(0)
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
    rep += 1

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
    elif dataset.lower() == 'hcp':
        dataset = 'hcp'
        
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

    if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
        vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        # compare the dim with the first file dim

        if vol.np.shape != dim:
            raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

            
        # to have a binary 3D structure
        dic_vol[subject_id] = (vol.np > 0).astype(int)
        sum_vol.np[:] += (vol.np > 0).astype(int) 
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

sum_vol = sum_vol / len(subject_id_list)
print(sum_vol, sum_vol.shape)


a_sum_vol = a.toAObject(sum_vol)
a_sum_vol.setPalette(minVal=0, absoluteMode=True)
wsum = a.createWindow('Sagittal', block=block)
wsum.addObjects(a_sum_vol)
rvol = a.fusionObjects(objects=[a_sum_vol], method='VolumeRenderingFusionMethod')
rvol.releaseAppRef()
# custom palette
n = len(subject_id_list)
pal = a.createPalette('VR-palette')
pal.header()['palette_gradients'] = f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
build_gradient(pal)
rvol.setPalette('VR-palette', minVal=0, absoluteMode=True)
pal2 = a.createPalette('slice-palette')
pal2.header()['palette_gradients'] = f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
build_gradient(pal2)
a_sum_vol.setPalette('slice-palette')
# rvol.palette().fill()
wvr = a.createWindow('3D', block=block)
wvr.addObjects(rvol)
#print(dic_vol[subject_id_list[0]].shape)
#print(np.count_nonzero(dic_vol[subject_id_list[0]]))

Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 8
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


no position could be read at 477, 21
Position : 15.9985, 39.8893, 30.1193, 0
Position : 15.9985, 39.8893, 30.1193, 0
Position : 15.9912, 69.7893, 1.3619, 0
Position : 15.9973, 61.1209, 30.7875, 0
no position could be read at 520, 136
Position : 15.997, 64.5184, 73.2273, 0
Position : 15.997, 71.1751, 69.8779, 0
Position : 15.997, 71.0977, 72.2557, 0
Position : 15.997, 71.673, 72.8699, 0
Position : 16.1375, 26.0942, 49.8344, 0
Position : 16.0368, 46.3805, 36.4938, 0
Position : 16.0114, 42.4504, 38.9599, 0
Position : 15.9994, 42.5725, 36.7169, 0
